In [2]:
import numpy as np
import pandas as pd
import CBSparserOData4
import re
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
def laad_uitslagenPC6(path):
    uitslagen = pd.read_csv(path,sep="|")
    uitslagen['PC6'] = uitslagen['postcode']
    uitslagen['PC4'] = uitslagen['postcode'].str[0:4]
    uitslagen.drop(['postcode','huisnummertoevoeging','letter'], inplace = True, axis=1)
    register = uitslagen[['stad','straatnaam','stembureau','postcode_stembureau','PC6','PC4']]
    
    # Voor het gemak brengen we de data terug naar PC6 niveau, want de CBS data heeft toch niet meer nauwkeurigheid.
    # Later kan worden gecorrigeerd voor verschillen tussen PC6 en de zone rondom een stembureau.
    # Voordat de data gegroepeerd wordt moeten tabellen voor het koppelen van tekstkolommen bewaard worden.
    # Want een groupby wist alle tekstkolommen.
    # verkiezing = demo['verkiezing'].values[0] # Deze waarde is nodig mochten meerdere verkiezingen tegelijk geladen worden.
    
    uitslagen=uitslagen.groupby('PC6').median()
    #uitslagen=uitslagen.groupby(['postcode_stembureau','straatnaam']).median()
    #uitslagen=uitslagen.groupby(['PC6','straatnaam']).median()
    
    return uitslagen, register

In [4]:
def laad_CBSdataPC6(path):
    demo = pd.read_csv(path, sep=",", index_col=0)
    demo = demo.groupby('PC6').median()
    
    return demo
    

In [5]:
onderzoek = pd.read_csv('CBS_PC6_selectie.csv', sep=",", index_col=0)
onderzoek.describe()

,INWONER,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,AANTAL_HH,GEM_HH_GR,WONING,WON_MRGEZ,UITKMINAOW,Perc_NW_migracht,Gemcode2019
count,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,137324.000000,156810.000000,156810.000000,156810.000000,124604.000000,157023.000000
mean,40.477074,19.937217,20.351189,5.521236,4.539825,11.357566,9.906256,5.455232,20.023277,2.084678,18.889133,10.397519,2.923155,20.658807,415.164422
std,37.923528,19.741885,19.367376,8.640256,11.503882,15.545825,10.536750,10.964677,22.160447,0.620100,19.956008,20.454487,7.824927,23.333988,240.324323
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.000000
25%,20.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.600000,10.000000,0.000000,0.000000,0.000000,228.000000
50%,35.000000,15.000000,20.000000,0.000000,0.000000,10.000000,10.000000,0.000000,20.000000,2.000000,15.000000,0.000000,0.000000,10.000000,363.000000
75%,50.000000,25.000000,25.000000,10.000000,5.000000,15.000000,15.000000,10.000000,25.000000,2.500000,25.000000,20.000000,5.000000,30.000000,546.000000
max,2020.000000,1385.000000,1145.000000,290.000000,1365.000000,880.000000,580.000000,335.000000,2020.000000,6.000000,800.000000,800.000000,1290.000000,100.000000,983.000000


Normaliseer met aantal inwoners: MAN, VROUW, INW_xxx, UITKMINAOW,
Normaliseer met aantal huishoudens: WON_MRGEZ, 
Laat weg: INWONER, AANTAL_HH, Perc_NW_migracht. Deze laatse kolom bevat veel missende waarden.

In [6]:
def join_PC6(uitslagen,demo):
    # Omdat de data per PC6 is gemeten is het logisch om eerst met de PC6 de databases te koppelen.
    return uitslagen.join(demo)

In [7]:
feature_select = ['MAN','VROUW','UITKMINAOW','GEM_HH_GR']
# 'WON_MRGEZ' delen door aantal huishoudens levert niet een fractie van aantal huishoudens(waarde tussen 0 en 1) op.
# Gebruik deze feature pas als we weten wat het betekent.

In [114]:
def normaliseer_PC6(df, features=None, dropNA=True, drempel = None):
    # verwijder ook NaN's ?
    
    # Lijsten van alle mogelijke features, opgedeeld in hoe ze verwerkt moeten worden
    INW = [] # Wordt gevuld met inwoners in leeftijdsgroep en features in overig_persoon
    perc = ['stemperc'] # Opkomst en percentages stemmen worden altijd toegevoegd.
    huish = ['WON_MRGEZ']
    overig_persoon = ['MAN','VROUW','UITKMINAOW']
    overig_normaal = ['GEM_HH_GR'] # Features die al genormaliseerd zijn.
    zwarte_lijst = ['Perc_NW_migracht'] # Teveel missende waarden in deze kolom
    
    # verwijder kolommen 
    def overbodig(lijst, checklist):
        for c in lijst:
            if c not in checklist:
                lijst.remove(c)
        return lijst
    
    if features:
        huish = overbodig(huish, features)
        overig_persoon = overbodig(overig_persoon, features)
        overig_normaal = overbodig(overig_normaal, features)
        
    if drempel:
        stuk = df[df['INWONER'] >= drempel]
        df = df.loc[ stuk.index ]
        
    for col in df.columns:
        if col in overig_persoon or re.search('^INW_',col):
            df[col] = df[col] / df['INWONER']
            INW.append(col)
        elif col in huish:
            df[col] = df[col] / df['AANTAL_HH']
        elif col in zwarte_lijst:
            df.drop(col, axis=1,inplace=True)
        elif re.search('percentage$',col):
            perc.append(col)
            
    if features:
        cols = INW+overig_normaal+huish+perc
        print(cols)
        df = df[cols]
        
    if dropNA:
        df = df.dropna(axis=0)
    
    return df

In [9]:
EP2019, register = laad_uitslagenPC6("adressen_gl_prioriteit/adressen_EP2019.csv")
EP2019.head()

C:\Users\DKuipers\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,nummer,lat,lon,stemperc,prioriteit,totaal,opkomst,D66_stemmen,GL_stemmen,PvdA_stemmen,PvdD_stemmen,SP_stemmen,DENK_stemmen,D66_percentage,GL_percentage,PvdA_percentage,PvdD_percentage,SP_percentage,DENK_percentage
PC6,,,,,,,,,,,,,,,,,,,
1011AB,111.0,52.377773,4.905684,0.796163,0.0,417.0,0.429675,51.0,153.0,80.0,35.0,5.0,8.0,0.122302,0.366906,0.191847,0.083933,0.011990,0.019185
1011AC,147.0,52.377385,4.908563,0.796163,0.0,417.0,0.429675,51.0,153.0,80.0,35.0,5.0,8.0,0.122302,0.366906,0.191847,0.083933,0.011990,0.019185
1011AD,1.0,52.376820,4.904492,0.741339,0.0,433.0,0.481914,54.0,118.0,102.0,36.0,11.0,0.0,0.124711,0.272517,0.235566,0.083141,0.025404,0.000000
1011AE,145.0,52.376399,4.910231,0.796163,0.0,417.0,0.429675,51.0,153.0,80.0,35.0,5.0,8.0,0.122302,0.366906,0.191847,0.083933,0.011990,0.019185
1011AG,98.5,52.375865,4.902948,0.741339,0.0,433.0,0.481914,54.0,118.0,102.0,36.0,11.0,0.0,0.124711,0.272517,0.235566,0.083141,0.025404,0.000000


In [10]:
demo = laad_CBSdataPC6('CBS_PC6_selectie.csv')
demo.head()

,INWONER,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,AANTAL_HH,GEM_HH_GR,WONING,WON_MRGEZ,UITKMINAOW,Perc_NW_migracht,Gemcode2019
PC6,,,,,,,,,,,,,,,
1011AB,5.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,1.4,0.0,0.0,0.0,NaN,363
1011AC,25.0,15.0,10.0,0.0,0.0,10.0,0.0,5.0,20.0,1.5,5.0,5.0,0.0,20.0,363
1011AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,363
1011AE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,363
1011AG,10.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,1.6,0.0,0.0,0.0,NaN,363


In [115]:
DataPC6 = join_PC6(EP2019,demo)
#print(DataPC6.columns)
print(DataPC6.head())
print(DataPC6.shape)

#test = DataPC6[DataPC6['INWONER'] >= 10]
#DataPC6.loc[test.index]

backup = DataPC6[['INWONER','prioriteit']]
DataPC6 = normaliseer_PC6(DataPC6,feature_select, drempel = 10)
DataPC6.describe()

        nummer        lat       lon  stemperc  prioriteit  totaal   opkomst  \
PC6                                                                           
1011AB   111.0  52.377773  4.905684  0.796163         0.0   417.0  0.429675   
1011AC   147.0  52.377385  4.908563  0.796163         0.0   417.0  0.429675   
1011AD     1.0  52.376820  4.904492  0.741339         0.0   433.0  0.481914   
1011AE   145.0  52.376399  4.910231  0.796163         0.0   417.0  0.429675   
1011AG    98.5  52.375865  4.902948  0.741339         0.0   433.0  0.481914   

        D66_stemmen  GL_stemmen  PvdA_stemmen  ...  INW_2544  INW_4564  \
PC6                                            ...                       
1011AB         51.0       153.0          80.0  ...       5.0       0.0   
1011AC         51.0       153.0          80.0  ...      10.0       0.0   
1011AD         54.0       118.0         102.0  ...       0.0       0.0   
1011AE         51.0       153.0          80.0  ...       0.0       0.0   
10

,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,UITKMINAOW,GEM_HH_GR,stemperc,D66_percentage,GL_percentage,PvdA_percentage,PvdD_percentage,SP_percentage,DENK_percentage
count,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000
mean,0.490464,0.502954,0.113560,0.094380,0.269591,0.240802,0.137576,0.066936,2.077614,0.566533,0.087225,0.160242,0.197828,0.052725,0.038969,0.029522
std,0.110360,0.110448,0.120404,0.133112,0.183897,0.148877,0.193655,0.113435,0.617161,0.117324,0.035926,0.075624,0.042880,0.019384,0.020908,0.058111
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.049505,0.005780,0.000000,0.014851,0.000000,0.000000,0.000000
25%,0.428571,0.444444,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000,1.600000,0.486239,0.059181,0.100592,0.169001,0.039062,0.023307,0.001855
50%,0.500000,0.500000,0.100000,0.000000,0.266667,0.250000,0.066667,0.000000,2.000000,0.561333,0.081744,0.142241,0.195041,0.051339,0.035398,0.008163
75%,0.555556,0.571429,0.200000,0.157895,0.384615,0.333333,0.200000,0.117647,2.500000,0.656842,0.112219,0.212316,0.223479,0.064572,0.050866,0.029412
max,1.000000,1.000000,0.800000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,0.907937,0.217445,0.438486,0.363853,0.184035,0.139130,0.585034


In [97]:
index = pd.read_csv('2019-cbs-pc6huisnr20190801_buurt/pc6hnr20190801_gwb.csv', sep=';')
#deze data vind je hier: https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer
index.head()

,PC6,Huisnummer,Buurt2019,Wijk2019,Gemeente2019
0,1011AB,105,3630400,36304,363
1,1011AB,106,3630400,36304,363
2,1011AB,107,3630400,36304,363
3,1011AB,110,3630400,36304,363
4,1011AB,112,3630400,36304,363


In [98]:
def prefixer(s, prefix, lengte):
    s = str(s)
    while len(s) < lengte:
        s = '0' + s
    s = prefix + s
    return s

def get_index():
    index = index.groupby('PC6').median()
    index['Buurt2019'] = index['Buurt2019'].astype('int').apply(lambda s: prefixer(s,'BU', 8))
    index['Wijk2019'] = index['Wijk2019'].astype('int').apply(lambda s: prefixer(s, 'WK', 6))
    index['Gemeente2019'] = index['Gemeente2019'].astype('int').apply(lambda s: prefixer(s, 'GM', 4))
    return index 

def get_prioriteit(df):
    if 'prioriteit' in df.columns:
        if 'PC6' in df.columns:
            return(df[['PC6','prioriteit']])
        else:
            return df.prioriteit

In [99]:
features = pd.read_csv("features_verzoek.csv",sep=";", index_col=0)
Data2017 = CBSparserOData4.parserOData4("83765NED","GM",features['Identifier'])
Data2017.head()
#Data 2019 kan niet geladen worden!

https://odata4.cbs.nl/CBS/83765NED$filter=contains(WijkenEnBuurten,'GM')
Download geslaagd.


,1010,1014800_1,1014800_2,1014800_3,1014850_2,1014850_3,1014850_4,1016030,1016040,1020,...,ZW10320_2,ZW10340,ZW25805_1,ZW25805_2,ZW25806_1,ZW25806_2,ZW25807,ZW25808,ZW25810_1,ZW25810_2
WijkenEnBuurten,,,,,,,,,,,,,,,,,,,,,
GM0003,5075.0,48.0,2830.0,1630.0,51.0,2140.0,1220.0,1733.0,1800.0,4997.0,...,1960.0,28.0,2560.0,1360.0,2650.0,1550.0,15.0,85.0,1790.0,960.0
GM0005,4599.0,68.0,2830.0,1680.0,31.0,2070.0,1280.0,1580.0,1526.0,4720.0,...,2020.0,14.0,2530.0,1380.0,2520.0,1510.0,14.0,86.0,1690.0,890.0
GM0007,3259.0,71.0,3520.0,2040.0,28.0,2370.0,1620.0,1179.0,1606.0,4294.0,...,2130.0,6.0,2230.0,1470.0,2400.0,1630.0,6.0,94.0,1810.0,1080.0
GM0009,3231.0,77.0,2890.0,1730.0,23.0,2100.0,1340.0,1176.0,987.0,3137.0,...,2000.0,9.0,2530.0,1440.0,2570.0,1610.0,15.0,85.0,1740.0,780.0
GM0010,10264.0,62.0,2890.0,1670.0,37.0,2160.0,1310.0,3585.0,3815.0,10894.0,...,2000.0,22.0,2650.0,1400.0,2660.0,1570.0,9.0,91.0,1750.0,1020.0


## Controle kwaliteit data
In kolommen met een fractie (tussen 0 en 1) zitten nog opvallend veel datapunten met (bijna) precies 0 en 1.
Dat is erg vreemd, zeker gezien het feit dat eerder de mediaan per PC6 is genomen. (Gemiddelden hebben geen betekenis omdat het gaat om dezelfde meting met steeds een ander adres)
In welke kolommen speelt dit probleem, en in welke mate? Wat kunnen we zeggen over deze datapunten? Is dat genoeg reden om deze weg te laten?
Zie ook ???


### Uitkomst:
'GEM_HH_GR' heeft als minimum 1 en maximum 6, en is dus betrouwbaar.


De meest realistische mogelijkheid is dat we een minimum hanteren voor het aantal inwoners of huishoudens dat een PC6 moet hebben om gebruikt te mogen worden. De grootte van één straat is daarbij een goede vuistregel.

In [116]:
DataPC6 = DataPC6.join(backup['INWONER'])
for col in DataPC6.columns[0:9]:
    MIN = min(DataPC6[col])
    mins = DataPC6[ DataPC6[col] == MIN ][col]
    MAX = max(DataPC6[col])
    maxs = DataPC6[ DataPC6[col] == MAX ][col]
    print("In {} komt {}, het minimum, {} maal voor. {}, het maximum, komt {} maal voor.".format(
        col,MIN,len(mins),MAX,len(maxs)))

In MAN komt 0.0, het minimum, 1510 maal voor. 1.0, het maximum, komt 319 maal voor.
In VROUW komt 0.0, het minimum, 1363 maal voor. 1.0, het maximum, komt 394 maal voor.
In INW_014 komt 0.0, het minimum, 58422 maal voor. 0.8, het maximum, komt 1 maal voor.
In INW_1524 komt 0.0, het minimum, 67047 maal voor. 1.0, het maximum, komt 121 maal voor.
In INW_2544 komt 0.0, het minimum, 22906 maal voor. 1.0, het maximum, komt 240 maal voor.
In INW_4564 komt 0.0, het minimum, 21298 maal voor. 1.0, het maximum, komt 56 maal voor.
In INW_65PL komt 0.0, het minimum, 62850 maal voor. 1.0, het maximum, komt 975 maal voor.
In UITKMINAOW komt 0.0, het minimum, 85531 maal voor. 1.0, het maximum, komt 12 maal voor.
In GEM_HH_GR komt 1.0, het minimum, 1463 maal voor. 6.0, het maximum, komt 1 maal voor.


In [101]:
C = 'VROUW'
MIN = min(DataPC6[C])
#mins = DataPC6[ DataPC6[C] <= MIN ]
MAX = max(DataPC6[C])
#maxs = DataPC6[ DataPC6[C] >= MAX ]
extreem1 = DataPC6[DataPC6[C] >= MAX][[C,'INWONER']]
extreem2 = DataPC6[DataPC6[C] <= MIN][[C,'INWONER']]
print(extreem1.describe(),'\n',extreem2.describe())

       VROUW     INWONER
count  465.0  465.000000
mean     1.0   11.204301
std      0.0    5.338860
min      1.0    5.000000
25%      1.0   10.000000
50%      1.0   10.000000
75%      1.0   10.000000
max      1.0   45.000000 
         VROUW      INWONER
count  1677.0  1677.000000
mean      0.0    10.822898
std       0.0     5.142040
min       0.0     5.000000
25%       0.0    10.000000
50%       0.0    10.000000
75%       0.0    10.000000
max       0.0    95.000000


Kolom MAN: meer dan driekwart van minimum waarden heeft 10 inwoners of minder, meer dan driekwart van maximum waarden heeft 15 inwoners of minder.
Kolom VROUW: meer dan driekwart van minimum waarden heeft 10 inwoners of minder, meer dan driekwart van maximum waarden heeft 15 inwoners of minder.

Help on function drop in module pandas.core.frame:

drop(self, labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')
    Drop specified labels from rows or columns.
    
    Remove rows or columns by specifying label names and corresponding
    axis, or by specifying directly index or column names. When using a
    multi-index, labels on different levels can be removed by specifying
    the level.
    
    Parameters
    ----------
    labels : single label or list-like
        Index or column labels to drop.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Whether to drop labels from the index (0 or 'index') or
        columns (1 or 'columns').
    index, columns : single label or list-like
        Alternative to specifying axis (``labels, axis=1``
        is equivalent to ``columns=labels``).
    
        .. versionadded:: 0.21.0
    level : int or level name, optional
        For MultiIndex, level from which the labels will be removed.

In [ ]:
# Met dank aan https://www.w3schools.com/python/python_ml_linear_regression.asp
plot_selectie = ['MAN','VROUW']
y_feature = 'GL_percentage'
fig, ax = plt.subplots( len(plot_selectie) )
kolommen = ['slope','intercept','r','p','std_err']
rijen = []

for n, feature in enumerate(plot_selectie):
    x = DataPC6[feature]
    slope, intercept, r, p, std_err = stats.linregress(x, DataPC6[y_feature])
    rij = pd.Series((slope,intercept,r,p,std_err), index = kolommen)
    rijen.append(rij)
    
    def linfunc(x):
        return slope*x + intercept
    linmodel = list(map(linfunc, x))
    ax[n].scatter(x,dataPC6[y_feature], s=0.1)

data = pd.DataFrame(rijen, index = plot_selectie)
plt.show()

Slimmer indelen:
Maak een functie die in de oorspronkelijke tabel informatie uit tekstkolommen ophaalt na het invoeren van een PC6 (van stembureau). Laat deze een kolom aanmaken die aangeeft of er straten (of adressen) zijn binnen dat kader die buiten die PC6 vallen. Bijvoorbeeld met een cijfer dat het aantal combinaties aangeeft.